# Masked word completion with BERT

In [1]:
pip install pyspellchecker

Note: you may need to restart the kernel to use updated packages.


In [2]:
import re
import torch
import logging
import numpy as np
import matplotlib.pyplot as plt

In [3]:
from transformers import BertTokenizer, BertModel, pipeline

In [4]:
from spellchecker import SpellChecker

In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)

In [6]:
unmasker = pipeline('fill-mask', model='bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## 1. 입력 문장 처리
input으로 문장이 들어오면 공백 단위로 자르고 각각의 단어를[MASK]로 바꿔서 문장을 만든다.<br>
예시 문장

발음이 헷갈릴법한 단어 위주로 선정<br>
동음이자어(heterograph): 동일한 발음을 가졌지만 철자가 다른 단어를 의미

1. their, there, they're<br>
They are there with their dogs. 그들은 그들의 개와 함께 거기에 있다.<br>
They are with there dogs.

In [487]:
text1 = "Dey are with their dogs."

2. want, won’t
I don't want to work at an office.저는 사무실에서 일하고 싶지 않아요.(사무직을 갖고 싶지 않아요)

In [9]:
text2 = "i don't won't to work at an office." # i don't want to work at an office.

3. walk, work
We went for a walk along the sand. 우리는 모래사장을 따라 산책을 나갔다.
we work along the road.
we walk along the saend.
we wilk along the road.

In [468]:
text1 = "we went for a walk along the sand." # We went for a walk along the sand.

4. lock, rock
She rammed the key into the lock. 그녀는 열쇠를 자물쇠 안으로 넣었다.
she rammed the key into the rock.
she lamed the key into the lock.
she rammed the ki into the lock.

In [237]:
text1 = "she lamed the key into the lock." # The height of the bicycle seat is adjustable.

5. sit, seat
Could you find me my seat, please? 제 자리가 어디인지 찾아 주시겠어요?
Could you find me my sit, please?
Could you fine me my seat, please?

In [337]:
text1 = "Could you fine me my seat, please?" # The height of the bicycle seat is adjustable.

6. Lose / loose	
The engine came loose from its mountings. 
엔진이 받침대에 고정된 부분이 헐거워졌다.

It’s hard to see how they can lose.  
그들이 어떻게 질 수 있는지 이해하기 어렵다.

In [377]:
text1 = "It's hard to ce how they can lose." # The height of the bicycle seat is adjustable.

7. principle / principal	
He acted out of expediency, not principle. 
그는 원칙이 아닌 편의성에 따라 행동을 했다.

The principal is a very busy woman. 
교장 선생님은 너무 바쁜 분이다

In [480]:
text1 = "He acted out of expediency, not principle."

8. cite / site / sight	
If you quote a writer in academic work, you must cite the source. 
과제나 논문에 어떤 작가의 글을 인용한다면 반드시 출처를 남겨야 한다.

The monument was moved bodily to a new site.  
그 기념비는 온전히 새 장소로 이전되었다.

She paled visibly at the sight of the police car.  
그녀는 경찰차를 보자 얼굴이 눈에 띄게 창백해졌다.

In [16]:
text8 = "The height of the bicycle sit is adjustable." # The height of the bicycle seat is adjustable.

9. Sew /sow		
Can you sew a button on for me?  
내 옷에 단추 좀 달아 주겠니?

You must sow before you can reap. 
수확하기 전에 씨를 뿌려야 한다[노력 없이는 성과도 없다].

In [17]:
text9 = "The height of the bicycle sit is adjustable." # The height of the bicycle seat is adjustable.

10. Know / no
I need to know around how many people are coming in order to make dinner reservations, so please let me know asap.  
저녁 예약하려면 대충 몇 명이 올지 알아야 돼. 그러니까 최대한 빨리 알려줘.

I know him by sight, but I don't know who he is.
저 사람 얼굴은 눈에 익은데 누군지는 몰라.

In [18]:
text10 = "The height of the bicycle sit is adjustable." # The height of the bicycle seat is adjustable.

11. Buy / bye / by	
You can buy brushes, paint, varnish and suchlike there.
거기에서는 붓, 페인트, 니스, 그러한 것들을 살 수 있다.

That jacket was a really good buy. 
그 재킷은 정말 잘 샀어

She said goodbye and thanked us for coming. 
그녀가 작별 인사를 하며 우리에게 와 줘서 고맙다고 했다.

Nocturnal animals sleep by day and hunt by night. 
야행성 동물은 낮에 자고 밤에 사냥을 한다.

In [19]:
text11 = "The height of the bicycle sit is adjustable." # The height of the bicycle seat is adjustable.

12. Base / bass
Can you take it easy on the bass drum there? 
베이스 드럼 좀 살살 칠 수 없겠니?

Unlike most bassists, he plays a six-string bass. 
대부분의 베이시스트와는 달리, 그는 줄이 6개인 베이스를 연주한다.

This tasty fish is as mild as sea bass, but with a somewhat firmer texture. 
맛이 기막힌 이 물고기는 살 조직은 단단하면서도 농어처럼 담백합니다.

In [20]:
text12 = "The height of the bicycle sit is adjustable." # The height of the bicycle seat is adjustable.

13. Tier / tear

We have introduced an extra tier of administration. 
우리는 행정 처리 단계를 추가로 한 단계 더 도입했다.

She was unable to tear her eyes away from him.  
그녀는 그에게서 눈을 뗄 수가 없었다.

A tear plopped down onto the page she was reading. 
그녀가 읽고 있던 책장 위로 눈물이 한 방울 툭 떨어졌다

In [525]:
text1 = "He acted out of xpedienci, not principle." # The height of the bicycle seat is adjustable.

In [526]:
text1[-1]

'.'

In [527]:
text_split = list(map(str,text1.lower().split()))
text_list = []
for i in range(len(text_split)):
    temp = list(text_split)
    temp[i] = '[MASK]'
    if i == len(text_split)-1:
        if text1[-1] == '?':
            temp[i] = '[MASK]?'
        else:
            temp[i] = '[MASK].'
    text_list.append(' '.join(map(str,temp)))
    print(' '.join(map(str,temp)))
text_split

[MASK] acted out of xpedienci, not principle.
he [MASK] out of xpedienci, not principle.
he acted [MASK] of xpedienci, not principle.
he acted out [MASK] xpedienci, not principle.
he acted out of [MASK] not principle.
he acted out of xpedienci, [MASK] principle.
he acted out of xpedienci, not [MASK].


['he', 'acted', 'out', 'of', 'xpedienci,', 'not', 'principle.']

## 2. 오타 검사를 위해 spell checker 사용
spell checker에서는 구두점이 영향을 주는 것으로 판단.
숫자나 영어를 제외한 문자는 제거(특수문자 등)

In [528]:
spell = SpellChecker()

In [529]:
for i in range(len(text_split)):
    new = re.sub('[-=+,#/\:.?!<>]',"",text_split[i])
    text_split[i] = new
text_split

['he', 'acted', 'out', 'of', 'xpedienci', 'not', 'principle']

In [530]:
misspelled = spell.unknown(text_split) # find those words that may be misspelled

miss_word = [] # 오류가 있는 단어 리스트 
miss_index = [] # 오류가 있는 단어 인덱스
candidate = {} # 가능한 후보 딕셔너리
for word in misspelled:
    miss_word.append(word)
    miss_index.append(text_split.index(word))
    candidate[word] = list(spell.candidates(word))
    
    
    print("가장 가능성이 높은 결과:",spell.correction(word)) # 철자가 틀린 단어에 대해 가장 가능성이 높은 결과를 반환
    print("수정 가능한 단어 후보:", spell.candidates(word)) # 철자가 틀린 단어에 대해 가능한 후보 세트를 반환

가장 가능성이 높은 결과: expediency
수정 가능한 단어 후보: {'expediency', 'expedience'}


In [531]:
candidate.get('wilk')

## 3. 단어간 유사도 구하기
Levenshtein Distance (Edit Distance): 두 문자열 간의 차이를 거리로 계산하는 방법<br>
하나의 문자열 S를 수정하여 다른 문자열 T로 변환시키고자 할 때, 삽입 (insert), 삭제 (delete), 대체 (substitute) 연산이 사용된다. <br>S를 T로 변환시키는데 필요한 최소의 편집 연산 횟수를 편집 거리라고 한다.

### 1) 편집 거리 구하는 함수

In [532]:
# 두 문자열 길이비교
def compareLength(s1, s2):
    if len(s1) >= len(s2):
        longstr, shortstr = s1, s2
    else:
        longstr, shortstr = s2, s1
    return longstr, shortstr

def getDistanceList(s1, s2):
    M = [[0] * (len(s1)+1) for _ in range(len(s1)+1)]# 길이가 긴 문자열을 기준으로 2차원 리스트 초기
    long = len(s1)
    short = len(s2)        
    for i in range(long+1):
        M[i][0] = i
    for j in range(short+1):
        M[0][j] = j
    for i in range(1, long+1):
        for j in range(1, short+1):
            if s1[i-1] == s2[j-1]:
                M[i][j] = M[i-1][j-1]
            else:
                M[i][j] = min(M[i-1][j], M[i-1][j-1], M[i][j-1]) + 1
    return M, M[long][short] 

### 2) 문자열 바꾸는 과정(삽입, 삭제, 변경)을 출력하는 함수

In [533]:
def getTrace(m, s1, s2):
    print("[", s2, "을(를)", s1, "로 바꾸는 과정 ]")
    i, j = len(s1) - 1, len(s2) - 1
    while not(i == 0 and j ==0):
        s = min(m[i-1][j], m[i-1][j-1], m[i][j-1])
        if s == m[i][j]:
            i -= 1
            j -= 1
        elif s == m[i][j-1]:
            print(s1[i-1] + "을(를) 삭제")
            j -= 1
        elif s == m[i-1][j-1]:
            print(s2[j-1] + "을(를) " +s1[i-1]  + "(으)로 변경")
            i -= 1
            j -= 1
        else:
            print(s1[i] + "을(를) 추가")
            i -= 1

## edit distance 구하는 예시

In [534]:
# 비교할 문자열 s1, s2
s1="caert"
s2="dadrt"
s1, s2 = compareLength(s1, s2) #문자열의 길이를 비교해서 s1에 긴문자열, s2에 짧은 문자열을 넣는다.

M, edit_distance = getDistanceList(s1, s2) # list, 편집거리 

print(s2, "와(과)", s1, "의 최소 편집 거리는", edit_distance , "이다.", end="\n\n")
getTrace(M, s1, s2) # 문자열 바꾸는 과정 출력

dadrt 와(과) caert 의 최소 편집 거리는 2 이다.

[ dadrt 을(를) caert 로 바꾸는 과정 ]
d을(를) e(으)로 변경
d을(를) c(으)로 변경


&nbsp;&nbsp; **(1) spell checker의 단어 후보 들을 edit distance가 작은 순서로 정렬 (일치하는 단어가 없을 경우를 대비해)<br>**
만약  spell checker에서 오타가 발견되면 문장을 masked lm에 넣어 spell checker에서 나온 단어 후보들을 찾아 score 값이 어떻게 되는지 확인해 가장 확률이 높은 것을 반환해 오타를 교정하고 문장을 완성시킨다.

spell checker에서 오타가 발견된 경우

In [535]:
search_candidate = candidate.get(original_token)
candidate.get(original_token)

In [536]:
if len(miss_index) > 0:
    for num in miss_index:
        masked_sentence = text_list[num]
        original_token = text_split[num]
        print("index number:",num,"\nmasked sentence:", masked_sentence, "\noriginal token:", original_token)
        search_candidate = candidate.get(original_token)
        candidate.get(original_token)
else:
    print('오타가 발견되지 않았습니다.')

index number: 4 
masked sentence: he acted out of [MASK] not principle. 
original token: xpedienci


In [537]:
fill = unmasker(masked_sentence, original_token = original_token, top_k=30500)#"[MASK] man worked as a carpenter."

In [538]:
result1 = []

for i in search_candidate:
    find = i
    print(find)
    for j in range(len(fill)):
        if find == fill[j].get('token_str'):
            print(fill[j])
            result1.append(fill[j])

expediency
expedience


### spell checker에서 나온 단어 후보들을 찾고 score를 기준으로 내림차순으로 정렬한 결과

In [539]:
result1 = sorted(result1, key = lambda x:-x['score'])
result1

[]

In [540]:
print('score가 가장 높은 단어:',result1[0]['token_str'])

IndexError: list index out of range

## 4. 정렬하지 않은 masked language model의 결과

In [541]:
#밑에 거랑 같은데 edit distance를 추가하지않은 그냥 간단한 코드
#for num in range(len(text_split)):
#    masked_sentence = text_list[num]
#    original_token = text_split[num]
#    print("index number:",num,"\nmasked sentence:", masked_sentence, "\noriginal token:", original_token)
#    print(unmasker(masked_sentence, original_token = original_token, top_k=30000)[:5])

In [542]:
fill2 = []
for num in range(len(text_split)):
    masked_sentence = text_list[num]
    original_token = text_split[num]
    #index",num," 
    print(masked_sentence, " original token:", original_token)
    fill2.append(unmasker(masked_sentence, original_token = original_token, top_k=30000))
    for i in range(len(fill2)):
        for j in range(len(fill2[i])):
            s1 = fill2[i][j].get('token_str')
            s2 = original_token
            s1, s2 = compareLength(s1, s2) #문자열의 길이를 비교해서 s1에 긴문자열, s2에 짧은 문자열을 넣는다.
    
            M, edit_distance = getDistanceList(s1, s2)
            #print(s2, "와(과)", s1, "의 최소 편집 거리는", edit_distance , "이다.", end="\n\n")
            fill2[i][j]['distance'] = edit_distance
    result2 = (fill2[num])
    for k in range(4):
        print(result2[k].get('token_str'),end='\t')
        print(result2[k].get('sequence'),end='\t')
        print(result2[k].get('score'),end='\t')
        print(result2[k].get('distance'))

[MASK] acted out of xpedienci, not principle.  original token: he
he	he acted out of xpedienci, not principle.	0.2805768549442291	0
they	they acted out of xpedienci, not principle.	0.27347809076309204	2
i	i acted out of xpedienci, not principle.	0.10436078906059265	2
she	she acted out of xpedienci, not principle.	0.08458615094423294	1
he [MASK] out of xpedienci, not principle.  original token: acted
was	he was out of xpedienci, not principle.	0.4058654308319092	5
worked	he worked out of xpedienci, not principle.	0.06393358111381531	4
is	he is out of xpedienci, not principle.	0.04533344879746437	5
lived	he lived out of xpedienci, not principle.	0.038377948105335236	3
he acted [MASK] of xpedienci, not principle.  original token: out
out	he acted out of xpedienci, not principle.	0.5507052540779114	0
because	he acted because of xpedienci, not principle.	0.17053009569644928	6
independently	he acted independently of xpedienci, not principle.	0.06727415323257446	12
instead	he acted instead of

## 5. masked language model의 결과에서 input word와 edit distance기준 오름차순 정렬 후 score 기준 내림차순 정렬한 결과

In [550]:
fill2 = []
for num in range(len(text_split)):
    masked_sentence = text_list[num]
    original_token = text_split[num]
    print(masked_sentence, " original token:", original_token)
    fill2.append(unmasker(masked_sentence, original_token = original_token, top_k=30000))
    for i in range(len(fill2)):
        for j in range(len(fill2[i])):
            s1 = fill2[i][j].get('token_str')
            s2 = original_token
            s1, s2 = compareLength(s1, s2) #문자열의 길이를 비교해서 s1에 긴문자열, s2에 짧은 문자열을 넣는다.
    
            M, edit_distance = getDistanceList(s1, s2)
            #print(s2, "와(과)", s1, "의 최소 편집 거리는", edit_distance , "이다.", end="\n\n")
            fill2[i][j]['distance'] = edit_distance
    result2 = sorted(fill2[num], key = lambda x:(x['distance'],-x['score']))
#    result2 = sorted(fill2[num], key = lambda x:(-x['score']))
    for k in range(4):
        print(result2[k].get('token_str'),end='\t')
        print(result2[k].get('sequence'),end='\t')
        print(result2[k].get('score'),end='\t')
        print(result2[k].get('distance'))

[MASK] acted out of xpedienci, not principle.  original token: he
he	he acted out of xpedienci, not principle.	0.2805768549442291	0
she	she acted out of xpedienci, not principle.	0.08458615094423294	1
we	we acted out of xpedienci, not principle.	0.04710749164223671	1
the	the acted out of xpedienci, not principle.	0.0005782983498647809	1
he [MASK] out of xpedienci, not principle.  original token: acted
acted	he acted out of xpedienci, not principle.	0.013383772224187851	0
ached	he ached out of xpedienci, not principle.	2.51587298407685e-05	1
wanted	he wanted out of xpedienci, not principle.	0.022763263434171677	2
opted	he opted out of xpedienci, not principle.	0.0020718658342957497	2
he acted [MASK] of xpedienci, not principle.  original token: out
out	he acted out of xpedienci, not principle.	0.5507052540779114	0
but	he acted but of xpedienci, not principle.	7.395708234980702e-05	1
opt	he acted opt of xpedienci, not principle.	2.4380885861319257e-06	1
gut	he acted gut of xpedienci, not

In [549]:
print('score가 가장 높은 단어:',result2[0]['token_str'])

score가 가장 높은 단어: principle


Masked lm에서 원하는 단어가 찾아지지 않은 경우 원래 단어가 어디에 있는지 겁색

In [552]:
masked_sentence = "he acted out of [MASK] not principle."
original_token = "xpedienci"
find = "expediency"
fill = unmasker(masked_sentence, original_token = original_token, top_k=30100)#"[MASK] man worked as a carpenter."

for j in range(len(fill)):
    if find == fill[j].get('token_str'):
        print(j,fill[j])

## To Do
### 1. 정확도 측정: 음성인식결과로 돌려보기
#### 1) deep speech에 음성 데이터 셋을 넣은 결과와 그 결과를 '이 코드'를 돌려서 차이 비교하기
- 결과를 코드에 어떻게 돌려서 score를 측정할 것인지 고민

#### 2) 유튜브에서 음성 가져오기
- 유튜브 음성 가져와서 텍스트로 변환하는 작업 어떻게 할지
   - 기존 자막 서비스 이용해서 텍스트로 추출
   - 추출한 텍스트를 '이 코드' 돌리고 차이가 있는지 비교

### 2. 리팩토링 & 연결


